# Figure 6, Panels E and F

In [1]:
# 0. Import
import os
import sys

import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn2
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

from scipy.io import mmread
from scipy.sparse import csr_matrix

In [2]:
import warnings
warnings.filterwarnings('ignore')
from celloracle import motif_analysis as ma
import celloracle as co
co.__version__

which: no R in (/apps/software/standard/core/jupyterlab/3.6.3-py3.11:/apps/software/standard/core/jupyterlab/3.6.3-py3.11/sbin:/apps/software/standard/core/jupyterlab/3.6.3-py3.11/bin:/home/apc5un/bedtools2/bin:/opt/mam/9.1.2/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/slurm/current/bin:/opt/singularity/current/bin:/opt/rci/bin:/share/rci_apps/common/bin:/share/resources/HPCtools:/opt/mam/current/bin:/opt/apptainer/current/bin)


'0.14.0'

In [3]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

## Make 12,000-edge Gephi-compatible files

Only doing this for E8.5 and E9 networks. Final hairballs made in Gephi.

In [4]:
links_wt_e85 = co.load_hdf5(file_path="./data/celloracle/e85/WT_cardiac-subset-links.celloracle.links")
links_wt_e9 = co.load_hdf5(file_path="./data/celloracle/e9/WT_cardiac-subset-links.celloracle.links")

links_ko_e85 = co.load_hdf5(file_path="./data/celloracle/e85/KO_cardiac-subset-links.celloracle.links")
links_ko_e9 = co.load_hdf5(file_path="./data/celloracle/e9/KO_cardiac-subset-links.celloracle.links")

In [5]:
links_wt_e85.filter_links(p=0.001, weight="coef_abs", threshold_number=12000)
links_wt_e85.filtered_links['IFT-CMs_WT'].sort_values('p')

,source,target,coef_mean,coef_abs,p,-logp
75646,Nkx2-5,Hspd1,-0.021870,0.021870,6.048811e-27,26.218330
130224,Gli1,Prom1,0.027861,0.027861,8.110916e-27,26.090930
103690,Nkx2-5,Myh6,0.166351,0.166351,5.653225e-26,25.247704
193268,Klf12,Zfp503,0.017696,0.017696,4.921638e-25,24.307890
166675,Klf12,Svep1,0.035120,0.035120,5.317990e-25,24.274253
...,...,...,...,...,...,...
171167,Sp5,Thbs3,0.000507,0.000507,9.989590e-04,3.000452
83294,Myc,Kcnq3,0.001513,0.001513,9.990317e-04,3.000421
43198,Klf3,Dkc1,-0.002243,0.002243,9.993684e-04,3.000274
130679,Atf6,Prrc2c,0.001036,0.001036,9.996912e-04,3.000134


In [6]:
links_dict = {}

for timepoint, wt_or_ko in [['85', 'WT'], ['85', 'KO'], ['9', 'WT'], ['9', 'KO']]:
    links = co.load_hdf5(file_path=f"./data/celloracle/e{timepoint}/{wt_or_ko}_cardiac-subset-links.celloracle.links")

    links.filter_links(p=0.001, weight="coef_abs", threshold_number=12500)
    filtered_df = links.filtered_links

    for cluster in filtered_df.keys():
        df = filtered_df[cluster]
        
        if wt_or_ko == 'KO':
            df = df.drop(df[df['source'] == 'Mef2c'].index)

        df = df.sort_values('coef_abs', ascending=False).iloc[:12000,:]

        sources = df['source'].unique()
        targets = df['target'].unique()

        # Create a zero matrix
        matrix = pd.DataFrame(np.zeros((len(sources), len(targets))), index=sources, columns=targets)

        # Populate the matrix with coef_mean values
        for index, row in df.iterrows():
            matrix.at[row['source'], row['target']] = row['coef_mean']

        matrix.to_csv(f"./data/celloracle/e{timepoint}/{wt_or_ko}_{cluster}_gephi.csv")